# MAIC Image Segmentation
<i>Workshop based off of https://www.tensorflow.org/tutorials/images/segmentation</i><br/><br/>
We will be using a neural network to identify the locations in an image that contain:
- A pixel belonging to a pet
- A pixel bordering the pet
- A pixel elsewhere<br/><br/>
Image segmentation is useful in a whole variety of tasks, MAIC has had research groups in the past focus on this area of study for identifying medical conditions.

In [ ]:
from IPython.display import clear_output
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf

We can download the pets dataset which already has all of the images with pixels selected for the three categories described above

In [ ]:
dataset, info = tfds.load('oxford_iiit_pet:3.*.*', with_info=True)

Our images contain RGB values between [0-255] but we want numbers to be between [0,1], so we'll have to scale them<br/><br/>We'll also shift our mask values to be 0, 1, 2 instead of 1, 2, 3. The mask is what contains the correct segmentation values

In [ ]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32) / 255.0
    input_mask -= 1
    return input_image, input_mask

In [ ]:
def load_image(datapoint):
    input_image = tf.image.resize(datapoint['image'], (128, 128))
    input_mask = tf.image.resize(
        datapoint['segmentation_mask'],
        (128, 128),
        method = tf.image.ResizeMethod.NEAREST_NEIGHBOR,
    )

    input_image, input_mask = normalize(input_image, input_mask)

    return input_image, input_mask

The dataset already has a train-test split so we can use those

In [ ]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train_images = dataset['train'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_images = dataset['test'].map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

Data augmentation is always good, this will do simple flips, however there are more and better ways to do data augmentation, check some out here: https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
class Augment(tf.keras.layers.Layer):
    def __init__(self, seed=42):
        super().__init__()
        # both use the same seed, so they'll make the same random changes.
        self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
        self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)

    def call(self, inputs, labels):
        inputs = self.augment_inputs(inputs)
        labels = self.augment_labels(labels)
        return inputs, labels

This will build an <bold>input pipeline</bold> this is specific to tensorflow but the general idea is a common one. This is great for when we work with large or complicated datasets, or we need to distribute datasets (across GPUs for instance).

In [ ]:
train_batches = (
    train_images
    .cache()
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE)
    .repeat()
    .map(Augment())
    .prefetch(buffer_size=tf.data.AUTOTUNE))

test_batches = test_images.batch(BATCH_SIZE)

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

We can take a look at what some of the inputs and their correct segmentations look like

In [ ]:
for images, masks in train_batches.take(2):
    sample_image, sample_mask = images[0], masks[0]
    display([sample_image, sample_mask])

The type of neural network we will use is called U-Net (learn more here: https://arxiv.org/pdf/1505.04597.pdf). They were first used for Biomedical image segmentation and work well for other types of image segmentation. This architecture uses an encoder-decoder setup. 

To help keep training times low for the workshop, we'll load a pretrained model (MobileNetV2, a CNN based architecture, learn more about it here: https://arxiv.org/pdf/1801.04381.pdf) for our encoder. And for the decoder, we'll use a simple upsampler with convolutional layers.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
base_model_outputs = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=base_model_outputs)

down_stack.trainable = False

In [ ]:
def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)

    result = tf.keras.Sequential()
    result.add(
        tf.keras.layers.Conv2DTranspose(filters, size, strides=2,
                                        padding='same',
                                        kernel_initializer=initializer,
                                        use_bias=False))

    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result

up_stack = [
    upsample(512, 3),  # 4x4 -> 8x8
    upsample(256, 3),  # 8x8 -> 16x16
    upsample(128, 3),  # 16x16 -> 32x32
    upsample(64, 3),   # 32x32 -> 64x64
]

In [ ]:
def unet_model(output_channels:int):
    inputs = tf.keras.layers.Input(shape=[128, 128, 3])

    # Downsampling through the model
    skips = down_stack(inputs)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = tf.keras.layers.Concatenate()
        x = concat([x, skip])

    # This is the last layer of the model
    last = tf.keras.layers.Conv2DTranspose(
        filters=output_channels, kernel_size=3, strides=2,
        padding='same')  #64x64 -> 128x128

    x = last(x)

    return tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
OUTPUT_CLASSES = 3

model = unet_model(output_channels=OUTPUT_CLASSES)
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.math.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [ ]:
def show_predictions(dataset=None, num=1):
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = model.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
                 create_mask(model.predict(sample_image[tf.newaxis, ...]))])

We can look at some predictions from an untrained model and see that it doesn't do so well

In [ ]:
show_predictions()

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

As we train, we should be able to see the predictions get better and better

In [ ]:
EPOCHS = 10
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_batches, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_batches,
                          callbacks=[DisplayCallback()])

We can try to identify overfitting by viewing our training history

In [ ]:
acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

plt.figure()
plt.plot(model_history.epoch, acc, 'r', label='Training Accuracy')
plt.plot(model_history.epoch, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
show_predictions(test_batches, 3)

# Challenge Problems
## Easy (1pt)
- Our training plot currently only shows accuracy, it can be more helpful sometimes to look at loss however, can you edit our accuracy plot to also include loss?
## Medium (2pt)
- As was mentioned earlier in the notebook, we're lacking data augmentation, add some more in, such as rotations and color shifts
## Hard (3pt)
- Our model currently has an issue called <strong>class imbalancing</strong>. You can learn some ways to mitigate it at the optional section of the following tutorial https://www.tensorflow.org/tutorials/images/segmentation#optional_imbalanced_classes_and_class_weights Try and impliment this here
## Not hard but worth more points (5pt)
- Upload and load an image of your pet (or a pet of someone you know).  Run inference on the image of your pet, and use that prediction to mask the image of your pet. Display that masked image with some sort of background. Upload your pets image, mask, and masked image to the creations and findings chat.